# Menghapus data yang nilainya 0

In [13]:
import pandas as pd
import numpy as np

# Baca dataset
df = pd.read_csv("../datasets/restaurant_dataset.csv")

print("Jumlah nilai 0 sebelum pembersihan:")
print((df == 0).sum())

# Kolom yang ingin diperbaiki
cols_to_fill = ['jumlah_mall', 'jumlah_minimarket', 'jumlah_taman', 'google_rating']

# Ganti 0 menjadi NaN hanya di kolom yang ingin diisi
df[cols_to_fill] = df[cols_to_fill].replace(0, np.nan)

# Isi NaN berdasarkan rata-rata per kecamatan (jika kolom 'kecamatan' ada)
for col in cols_to_fill:
    df[col] = df.groupby('kecamatan')[col].transform(lambda x: x.fillna(x.mean()))
    # Fallback ke rata-rata global jika masih ada NaN
    df[col].fillna(df[col].mean(), inplace=True)

# Hapus baris dengan nilai 0 di 'Jumlah Penduduk'
if 'Jumlah Penduduk' in df.columns and (df['Jumlah Penduduk'] == 0).any():
    print("Menghapus baris dengan nilai 0 pada kolom 'Jumlah Penduduk'.")
    df = df[df['Jumlah Penduduk'] != 0]

# Cek apakah masih ada nilai null
print("\nJumlah nilai NULL setelah pembersihan:")
print(df.isnull().sum())

# Simpan ke file baru
df.to_csv("../datasets/restaurant_dataset_cleaned.csv", index=False)


Jumlah nilai 0 sebelum pembersihan:
nama                       0
alamat                     0
kecamatan                  0
google_rating              0
price_range_rupiah         0
business_status            0
processed_at               0
Jumlah Penduduk            1
Luas Wilayah (km²)         0
Kepadatan (jiwa/km²)       0
jumlah_mall             2493
jumlah_minimarket          1
jumlah_taman              32
dtype: int64
Menghapus baris dengan nilai 0 pada kolom 'Jumlah Penduduk'.

Jumlah nilai NULL setelah pembersihan:
nama                       0
alamat                     0
kecamatan                  0
google_rating              0
price_range_rupiah      3373
business_status            1
processed_at               0
Jumlah Penduduk            0
Luas Wilayah (km²)         0
Kepadatan (jiwa/km²)       0
jumlah_mall                0
jumlah_minimarket          0
jumlah_taman               0
dtype: int64


C:\Users\ridho\AppData\Local\Temp\ipykernel_18288\763595639.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
C:\Users\ridho\AppData\Local\Temp\ipykernel_18288\763595639.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

In [ ]:
df_clean = pd.read_csv("../datasets/restaurant_dataset_cleaned.csv")

df_clean.drop(columns=['business_status','processed_at'], inplace=True)

df_clean.isna().sum()

nama                       0
alamat                     0
kecamatan                  0
google_rating              0
price_range_rupiah      3373
Jumlah Penduduk            0
Luas Wilayah (km²)         0
Kepadatan (jiwa/km²)       0
jumlah_mall                0
jumlah_minimarket          0
jumlah_taman               0
dtype: int64

In [24]:
df_price_range = df['price_range_rupiah']

df_price_range.isna()

# df_clean.info()

0       False
1        True
2        True
3       False
4       False
        ...  
5111    False
5112    False
5113    False
5114    False
5115     True
Name: price_range_rupiah, Length: 5115, dtype: bool